In [ ]:
import duckdb
import pandas as pd
con = duckdb.connect("/tmp/dbt.duckdb")

In [ ]:
table_schema = 'main_main'
table = 'aadsc_stg_clinical'

## DB and Table general info

In [ ]:
# Fetch tables in the specified schema
tables = con.execute(
    f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{table_schema}'"
).fetchall()
tables

In [ ]:
# Fetch column names of the table
query = f"SELECT name FROM pragma_table_info('{table_schema}.{table}') LIMIT 15"
column_names = con.execute(query).fetchall()
column_names

## Sanity checks

In [ ]:
view_all = pd.DataFrame(con.execute(
        f"SELECT * FROM {table_schema}.{table} LIMIT 50"
    ).fetchall())
view_all

## Investigate columns

In [ ]:

# Understanding the conditions that mention 'age'. 
# I believe these cols represent conditions that were diagnosed,
#   and the positive assertion was collected via their diagnosis age.

dist_age_cols = con.execute(
        f"SELECT DISTINCT cardiac_pfo_age,cardiac_pda_age,skin_acne_age,skin_alopecia_age,cardiac_asd_age,cardiac_tof_age,cardiac_vsd_age FROM {table_schema}.{table}"
    ).fetchall()
# dist_age_cols

In [ ]:
# Investigate the extraction_date col
# Seems to be the data extraction date, not necessary for transformations.

extraction_date = con.execute(
        f"SELECT distinct extraction_date FROM {table_schema}.{table}"
    ).fetchall()

In [ ]:
# Ensure understanding of the bmi field
# Catch where the calculation of bmi doesn't seem correct.

bmi_issues = con.execute(
f"SELECT height, weight, bmi, \
       (cast(weight as float) / (cast(height as float) * cast(height as float))) * 703 AS calculated_bmi, \
       cast(bmi as float) AS stored_bmi \
FROM {table_schema}.{table} \
WHERE (cast(weight as float) / (cast(height as float) * cast(height as float))) * 703 NOT BETWEEN cast(bmi as float) - 1 AND cast(bmi as float) + 1;").fetchall()
bmi_issues

## Notes 

Copy results to a csv <br>
    ```
        to_csv = con.execute( 
        f"COPY (SELECT DISTINCT cardiac_pfo_age,cardiac_pda_age,skin_acne_age,skin_alopecia_age,cardiac_asd_age,cardiac_tof_age,cardiac_vsd_age FROM {table_schema}.{table}) TO 'output_src.csv' (HEADER, DELIMITER ',')"
    ).fetchall()
    ```